In [ ]:
#最低限必要な情報:物件名・住所・間取り・家賃・階数
#希望条件：都内で利便性の高い場所・予算20万円程度・2人暮らし2K以上
#検索条件：JR山手線沿線・予算25万円以下・2K以上の間取り

In [ ]:
# STEP１：全ページの物件情報を全て取ってくる

# 必要なライブラリのインポート
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
from tqdm import tqdm #for文の進捗確認

# 最後のページの数値を取得する
root_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&co=1&ra=013&et=9999999&shkr1=03&ar=030&bs=040&ct=25.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&rn=0005&shkr4=03&cb=0.0&page={}'
res = requests.get(root_url.format(1))
res.encoding = 'utf-8'
soup = BeautifulSoup(res.text, 'html.parser')
total_page = int(soup.find('ol', class_='pagination-parts').find_all('li')[-1].text)

#物件情報を1件ずつ格納するための空のリストを用意しておく
d_list = []

# 正常にHTMLの情報が取得できれば以下のコードを実行
if res.status_code == 200:

    # 物件一覧ページをfor文で1ページずつループ
    for i in tqdm(range(1, total_page+1)):
        root_url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?fw2=&mt=9999999&cn=9999999&co=1&ra=013&et=9999999&shkr1=03&ar=030&bs=040&ct=25.0&shkr3=03&shkr2=03&mb=0&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&rn=0005&shkr4=03&cb=0.0&page={}'
        res = requests.get(root_url.format(i))
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')
        time.sleep(1)

        # 物件一覧ページの物件情報をfor文で1件ずつループ
        contents = soup.find_all('div', class_='cassetteitem')
        for content in contents:

            # 物件情報をdetailに格納する
            detail = content.find('div', class_='cassetteitem-detail')
            # 物件名
            name = detail.find('div', class_='cassetteitem_content-title').text
            # 住所
            address = detail.find('li', class_='cassetteitem_detail-col1').text
            # アクセス
            access1 = detail.find_all('div', class_='cassetteitem_detail-text')[0].text
            access2 = detail.find_all('div', class_='cassetteitem_detail-text')[1].text
            access3 = detail.find_all('div', class_='cassetteitem_detail-text')[2].text
            # 築年数/最上階
            age = detail.find('li', class_='cassetteitem_detail-col3').find_all('div')[0].text
            height = detail.find('li', class_='cassetteitem_detail-col3').find_all('div')[1].text

            # 物件情報の中で、部屋情報をfor文で1件ずつループ
            table = content.find('table', class_='cassetteitem_other')
            tr_tags = table.find_all('tr', class_='js-cassette_link')
            for tr_tag in tr_tags:

                # 部屋情報
                floor, price, first_fee, capacity = tr_tag.find_all('td')[2:6]
                floor = floor.text.replace('\r','').replace('\n','').replace('\t','')
                rent, admin = price.find_all('li')
                deposit, gratuity = first_fee.find_all('li')
                madori, menseki = capacity.find_all('li')

                # １つの部屋情報をまとめて辞書dに格納
                d = {
                    'name':name,
                    'address':address,
                    'access1':access1,
                    'access2':access2,
                    'access3':access3,
                    'age':age,
                    'height':height,
                    'floor':floor,
                    'rent':rent.text,
                    'admin':admin.text,
                    'deposit':deposit.text,
                    'gratuity':gratuity.text,
                    'madori':madori.text,
                    'menseki':menseki.text,
                }

                # 部屋情報をd_listに1件ずつ追加
                d_list.append(d)

#完成したリストはDataFrameに格納
df = pd.DataFrame(d_list)

In [ ]:
# STEP2：dfをスプレッドシートに格納する

# 必要なライブラリのインポート
import gspread
from google.oauth2.service_account import Credentials
from gspread_dataframe import set_with_dataframe

# 認証のためのAPIキーを含むjsonファイル
google_spreadsheet_api = '自分のjsonファイルのパスを指定'
# 認証のためのアクセス先
scopes = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# 認証情報をcredentialsに代入
credentials = Credentials.from_service_account_file(
    google_spreadsheet_api,
    scopes=scopes
)

# 認証情報を使ってGoogleSpreadsheetにアクセス
gs = gspread.authorize(credentials)

# 対象のスプレッドシートのファイルとシートを指定。この時アクセス権限注意。
SP_SHEET_KEY = '自分のスプレッドシートのキーを指定'
workbook = gs.open_by_key(SP_SHEET_KEY)
worksheet = workbook.worksheet('自分のシートを指定')

# 対象のシートにdfのデータを記入。但しindexは除く。
set_with_dataframe(worksheet, df, include_index=False)